In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import os

DIVERSITY_UTILITY = float(os.environ.get('DIVERSITY_UTILITY'))
FRAC_ADMIT = float(os.environ.get('FRAC_ADMIT'))

In [2]:
print(DIVERSITY_UTILITY), print(FRAC_ADMIT)

0.25
0.25


(None, None)

In [3]:
###
## Load dataframe where each row is a single applicant
###

df = pd.read_csv('./df_test.csv')
df['key'] = df['R'].astype(str) + "_"  + df['T'].astype(str)

###
## Estimate E[Y(1)|T, A]
###

df_train = pd.read_csv('./df_train.csv')
df_stratum_utility = df_train[['R','T','Y']].groupby(['R','T']).mean().reset_index()
df_stratum_utility['stratum_utility'] = (df_stratum_utility['Y'] + DIVERSITY_UTILITY * df_stratum_utility['R']).round(2)
df_stratum_utility['key'] = df_stratum_utility['R'].astype(str) + "_" + df_stratum_utility['T'].astype(str)

df = df.merge(df_stratum_utility[['stratum_utility','key']], on='key')
df['ml_outcomes'] = df['stratum_utility']

In [4]:
##
# R = race, T = test score, ml_outcomes = expected utility from admitting, 
# ml_outcomes_{} counterfactual utility given race 
##

df[['R','T','ml_outcomes','T_minority','T_majority','Y']].sort_values(by='ml_outcomes')



,R,T,ml_outcomes,T_minority,T_majority,Y
999835,0,11,0.00,11,11,0
999838,0,11,0.00,7,11,0
999837,0,11,0.00,8,11,0
999836,0,11,0.00,11,11,0
999886,0,12,0.02,11,12,0
...,...,...,...,...,...,...
999913,1,98,1.25,98,115,1
999914,1,98,1.25,98,115,1
999925,1,109,1.25,109,126,1
998845,1,97,1.25,97,112,1


In [5]:
#df['ml_outcomes'] = df['ml_outcomes'] + 1*df['R']

In [6]:
###
## Get total utility from admitting people in a stratum 
###

df_ = df[['R','T','ml_outcomes']].groupby(['R','T']).sum().reset_index()

In [7]:
###
## Get number of people in each stratum
###

df_count = df[['R','T','ml_outcomes']].groupby(['R','T']).count().reset_index()
df_count.columns = ['R','T','Count']
df_count['N'] = df_count['Count']

In [8]:
###
## Merge summary tables to get one table with Race, Test Score, SUM(Utility), COUNT(applicants) per stratum
###

dff = df_.merge(df_count[['N']],left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False).reset_index().sort_values(by='index').reset_index()

In [9]:
# final info table
dff.sort_values(by='ml_outcomes')

,level_0,index,R,T,ml_outcomes,N
2,221,2,0,11,0.00,4
1,219,1,0,10,0.08,1
3,220,3,0,12,0.08,4
0,218,0,0,9,0.10,1
120,217,120,1,8,0.32,1
...,...,...,...,...,...,...
51,4,51,0,60,10970.40,18284
47,3,47,0,56,10985.52,21126
48,2,48,0,57,11030.04,20426
50,1,50,0,59,11169.64,19258


### Setup optimization problem 

In [10]:
from ortools.linear_solver import pywraplp


In [11]:
solver = pywraplp.Solver.CreateSolver('GLOP')


In [12]:
dff

,level_0,index,R,T,ml_outcomes,N
0,218,0,0,9,0.10,1
1,219,1,0,10,0.08,1
2,221,2,0,11,0.00,4
3,220,3,0,12,0.08,4
4,211,4,0,13,1.00,10
...,...,...,...,...,...,...
217,203,217,1,107,1.25,1
218,204,218,1,109,1.25,1
219,205,219,1,111,1.25,1
220,212,220,1,113,0.92,1


In [13]:
applicant_stratum = []
vars_cache = {}

# Objective: Maximize the expected utility of the admitted students
objective = solver.Objective()

# For each stratum
for ix, row in dff.iterrows():
    # probability of admission
    numvar = solver.NumVar(0.0, 1.0, str(ix))
    
    # store variable by index, and also by stratum R, T
    applicant_stratum.append(numvar)
    vars_cache[(row['R'],row['T'])] = numvar
    
    # Benefit of admitting people is total utility in that stratum
    objective.SetCoefficient(applicant_stratum[ix], float(row['ml_outcomes']))
objective.SetMaximization()


In [14]:
# Currently we have no constraints 
solver.NumConstraints()

0

In [15]:
# Constraint: At most K applicants
K = int(len(df)*FRAC_ADMIT)
print(K)
admit_quota = solver.Constraint(0, K)

# Total admits cannot exceed K 
for ix, row in dff.iterrows():
    admit_quota.SetCoefficient(applicant_stratum[ix], float(row['N']))

249999


In [16]:
# Now we have one constraint
solver.NumConstraints()

1

## Solve linear program

In [17]:
solver.ABNORMAL

4

In [18]:
status = solver.Solve()


In [19]:
status

0

In [20]:
solver.OPTIMAL

0

In [21]:
row = []
admit = []

for i in applicant_stratum:
    row.append(int(str(i)))
    admit.append(i.solution_value())

df_decisions = pd.DataFrame({'row_id':row,'decision':admit})

In [22]:
df_decisions

,row_id,decision
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0
...,...,...
217,217,1.0
218,218,1.0
219,219,1.0
220,220,1.0


In [23]:
dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)

,level_0,index,R,T,ml_outcomes,N,row_id,decision
49,0,49,0,58,11233.60,20060,49,0.0
50,1,50,0,59,11169.64,19258,50,0.0
48,2,48,0,57,11030.04,20426,48,0.0
47,3,47,0,56,10985.52,21126,47,0.0
51,4,51,0,60,10970.40,18284,51,0.0
...,...,...,...,...,...,...,...,...
120,217,120,1,8,0.32,1,120,0.0
0,218,0,0,9,0.10,1,0,0.0
1,219,1,0,10,0.08,1,1,0.0
3,220,3,0,12,0.08,4,3,0.0


In [24]:
xxx_ = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx_.sort_values(by='T')

,level_0,index,R,T,ml_outcomes,N,row_id,decision
119,215,119,1,7,0.50,2,119,0.0
120,217,120,1,8,0.32,1,120,0.0
0,218,0,0,9,0.10,1,0,0.0
121,207,121,1,9,1.00,4,121,0.0
1,219,1,0,10,0.08,1,1,0.0
...,...,...,...,...,...,...,...,...
114,200,114,0,123,2.00,2,114,1.0
115,209,115,0,124,1.00,1,115,1.0
116,199,116,0,125,2.00,2,116,1.0
117,208,117,0,127,1.00,1,117,1.0


In [25]:
xxx = dff.merge(df_decisions,left_index=True,right_index=True).sort_values(by='ml_outcomes',ascending=False)
xxx

,level_0,index,R,T,ml_outcomes,N,row_id,decision
49,0,49,0,58,11233.60,20060,49,0.0
50,1,50,0,59,11169.64,19258,50,0.0
48,2,48,0,57,11030.04,20426,48,0.0
47,3,47,0,56,10985.52,21126,47,0.0
51,4,51,0,60,10970.40,18284,51,0.0
...,...,...,...,...,...,...,...,...
120,217,120,1,8,0.32,1,120,0.0
0,218,0,0,9,0.10,1,0,0.0
1,219,1,0,10,0.08,1,1,0.0
3,220,3,0,12,0.08,4,3,0.0


In [26]:
len(xxx_)

222

In [27]:
xxx

,level_0,index,R,T,ml_outcomes,N,row_id,decision
49,0,49,0,58,11233.60,20060,49,0.0
50,1,50,0,59,11169.64,19258,50,0.0
48,2,48,0,57,11030.04,20426,48,0.0
47,3,47,0,56,10985.52,21126,47,0.0
51,4,51,0,60,10970.40,18284,51,0.0
...,...,...,...,...,...,...,...,...
120,217,120,1,8,0.32,1,120,0.0
0,218,0,0,9,0.10,1,0,0.0
1,219,1,0,10,0.08,1,1,0.0
3,220,3,0,12,0.08,4,3,0.0


In [28]:
(xxx['ml_outcomes']*xxx['decision']).sum()

197363.74000000002

In [29]:
def maxMLOutcomes(frac_minority=0.01):
    minority_admits = int(K*frac_minority)
    majority_admits = int(K*(1-frac_minority))
    
    b = df[df['R']==1].sort_values(by='ml_outcomes',ascending=False).head(n=minority_admits)
    w = df[df['R']==0].sort_values(by='ml_outcomes',ascending=False).head(n=majority_admits)
    
    return b['Y'].sum() + w['Y'].sum()    

In [30]:
def maxMLOutcomesminority(frac_minority=0.01):
    minority_admits = int(K*frac_minority)
    majority_admits = int(K*(1-frac_minority))
    
    b = df[df['R']==1].sort_values(by='ml_outcomes',ascending=False).head(n=minority_admits)
    w = df[df['R']==0].sort_values(by='ml_outcomes',ascending=False).head(n=majority_admits)
    
    return b['Y'].sum()   

In [31]:
df['key'] = df['R'].astype(str)+'_'+df['T'].astype(str)
xxx['key'] = xxx['R'].astype(str)+'_'+xxx['T'].astype(str)

In [32]:
len(xxx)

222

In [33]:
admit_decisions = df.merge(xxx,how='left',on='key')
admit_decisions['decision_random'] = pd.Series([random.random() for x in range(0,len(admit_decisions))]) < FRAC_ADMIT


FRAC_minority_POLICY = (admit_decisions['R_y'] * admit_decisions['decision']).sum()/admit_decisions['decision'].sum()
SUM_BP_POLICY = ( admit_decisions['Y'] * admit_decisions['decision']).sum()

FRAC_RANDOM_POLICY = (admit_decisions['R_y'] * admit_decisions['decision_random']).sum()/admit_decisions['decision_random'].sum()
SUM_BP_RAND_POLICY = ( admit_decisions['Y'] * admit_decisions['decision_random']).sum()


In [34]:
file = open('./lp_results.csv','w')
file.write('{}\t{}\t{}\n'.format('Policy Name','Frac Diverse Candidates','Total Graduated'))
file.write('{}\t{}\t{}\n'.format('Random',str(FRAC_RANDOM_POLICY),str(SUM_BP_RAND_POLICY)))
file.write('{}\t{}\t{}\n'.format('Maximize Utility',str(FRAC_minority_POLICY),str(SUM_BP_POLICY)))
file.close()
